# 仕組み
1、評価と選定
３要素（駒の価値、成るまでの距離、王（玉）までの距離　）でどの駒

# 各オブジェクト説明

kind = {0:"ou", 1:"gyoku", 2:"hisha", 3:"kaku",4:"kin", 5:"gin", 6:"keima", 7:"kyousha", 8:"hu"}　#駒の種類を特定 （自分は王、相手は玉）

way = {"ou":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]], 
       "gyoku":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]], 
       "hisha":[[-1, 0], [0, 1], [1, 0], [0, -1]], 
       "kaku":[[-1, 1], [1, 1], [1, -1], [-1, -1]], 
       "kin":[[-1, 1], [0, 1], [1, 1], [1, 0], [0, -1], [-1, 0]], 
       "gin":[[-1, 1], [0, 1], [1, 1], [1, -1], [-1, -1]], 
       "keima":[[-1, 2], [1, 2]], 
       "kyousha":[[0, 1]], 
       "hu":[[0, 1]], 
       "ryu":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]], 
       "uma":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]],
      } #各駒の進む方向

value_k = {"ou":[8, 8], "gyoku":[8, 8], "hisha":[12, 16], "kaku":[12, 16],"kin":[6, 6], "gin":[5, 6], "keima":[6, 6], "kyousha":[3, 6], "hu":[1, 6]} #各駒の評価係数
value_p = {"first":1, "second":2, "third":3} #各列の評価係数（相手陣地に近いほど高い）
value_d = {"first":2, "second":1} #王との距離の評価係数
end_f = 0 #対局終了フラグ
player1 = [0, 2, 4] #各プレーヤの持ち駒
player2 = [3, 1, 6]

stage = {0:[1, 0], 2:[0, 1], 4:[2, 0], 3:[0, 2], 1:[1, 2], 6:[2, 2]} #初期配置


# memo 
駒にプレイヤの識別タグ？を付けなければいけない
将来DBに駒情報とかのパラメータを置くということを考えながら設計する。

In [62]:
import numpy as np

In [63]:
kind = {0:"ou", 1:"gyoku", 2:"hisha", 3:"kaku",4:"kin", 5:"gin", 6:"keima", 7:"kyousha", 8:"hu"}

way = {"ou":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]], 
       "gyoku":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]], 
       "hisha":[[-1, 0], [0, 1], [1, 0], [0, -1]], 
       "kaku":[[-1, 1], [1, 1], [1, -1], [-1, -1]], 
       "kin":[[-1, 1], [0, 1], [1, 1], [1, 0], [0, -1], [-1, 0]], 
       "gin":[[-1, 1], [0, 1], [1, 1], [1, -1], [-1, -1]], 
       "keima":[[-1, 2], [1, 2]], 
       "kyousha":[[0, 1]], 
       "hu":[[0, 1]], 
       "ryu":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]], 
       "uma":[[-1, 1], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0]],
      } 

value_k = {"ou":[8, 8], "gyoku":[8, 8], "hisha":[12, 16], "kaku":[12, 16],"kin":[6, 6], "gin":[5, 6], "keima":[6, 6], "kyousha":[3, 6], "hu":[1, 6]}
value_l = {"first":1, "second":2, "third":3} 
value_d = {"first":2, "second":1}
end_f = 0
player1 = [0, 2, 4] #各プレーヤの持ち駒
player2 = [3, 1, 6]

stage = {0:[1, 0], 2:[0, 1], 4:[2, 0], 3:[0, 2], 1:[1, 2], 6:[2, 2]} #初期盤面（自分が下）後で自分とcpの駒の区別をつけられるようにしなきゃだめ

In [128]:
def rob(player, nextpos, stage):
    enemyplaces = {}
    for i in player:
        enemyplaces[i] = stage[i]
    for i in enemyplaces.keys():
        print("robablepos", i)
        if enemyplaces[i] == nextpos:
            return i
    return False 
    

In [129]:
print(rob(player1, [1, 0], stage))

robablepos 0
0


In [91]:
def evaluation_r(k, l, d, r) : #点数の合計をだすよ～
    if r != false
    return value_k[kind[k]][0] + value_l[l] + value_d[d] + value_k[kind[r]][0]#まだ成ることは考えてないよ～（value_k[kind[0]][0]の０のとこ）

In [92]:
def evaluation_n(k, l, d) : #点数の合計をだすよ～
    return value_k[kind[k]][0] + value_l[l] + value_d[d]#まだ成ることは考えてないよ～（value_k[kind[0]][0]の０のとこ）

In [65]:
def lin(me): #列の点数を出すよ～
    if me[1] == 2:
        return "third"
    elif me[1] == 1:
        return "second"
    elif me[1] == 0:
        return "first"

In [123]:
def dist(me, king): #王までの距離の点数を出すよ～
    dist = np.sqrt((king[0] - me[0]) ** 2 + (king[1] - me[1]) ** 2)
    if dist < 2 :
        return "first"
    elif dist >= 2 and dist <= 4 :
        return "second"

In [124]:
def cp_simu(player2):
    global points
    for piece in player2: #pieceを選択
        print("*piece=", piece)
        for w in way[kind[piece]]: #wayを選択
            print("way=", w)
            reverse_w = list(map(lambda x : x * -1, w))
            position = np.array(stage[piece]) #選んだ駒の現在の座標
            next_position = position + np.array(reverse_w) #進んだ先の座標
            l_next_position = next_position.tolist() #numpyの配列と標準のリストの形が違うからinとかやってもダメ臭い=>ndarray.tolist()使おうや
            print(list(map(lambda x:x >= 0 and x <= 2, l_next_position)))
            if all(map(lambda x:x >= 0 and x <= 2, l_next_position)) == True: #進んだ先が盤面内に収まっているか
                if l_next_position not in stage.values(): #進んだ先に駒がなかった場合
                    king_position = stage[0] #自分は王だから王のポジションを特定する
                    line = str(lin(next_position)) #列の評価係数(function_lin)
                    print("line=", line)
                    distance = str(dist(l_next_position, king_position)) #距離の評価係数(function_dist)
                    print("l_next_position=", l_next_position)
                    print("distance=", distance)
                    point = evaluation_n(piece, line, distance)
                    print("*point=", point) #手の駒の能力×列×距離の点数(function_evaluation)
                    points.append([piece, point, l_next_position])
                else:
                    print("もうおるやんけ！")#進んだ先が相手の駒だった場合は点数加算＆駒の位置の得点も加算
                    robable_piece = rob(player1, l_next_position, stage) #取れる駒の種類(function_rob)
                    if robable_piece == False:
                        print("味方やんけ")
                    else: #進んだ先の駒が敵駒だったばあい
                        print("robable_piece=", robable_piece)
                        print("line=", line)
                        print("l_next_position=", l_next_position)
                        print("distance=", distance)
                        point = evaluation_r(piece, line, distance, robable_piece)
                        print("*駒取った+point=", point) #手の駒の能力+列+距離の点数+取った駒の能力(function_evaluation)
                        points.append([piece, point, l_next_position])
                    
            else:
                print("進めないよバーカ")
    
    return "points=", points
            

In [130]:
points =[]
print (cp_simu(player2))

*piece= 3
way= [-1, 1]
[True, True]
line= second
l_next_position= [1, 1]
distance= first
*point= 16
way= [1, 1]
[False, True]
進めないよバーカ
way= [1, -1]
[False, False]
進めないよバーカ
way= [-1, -1]
[True, False]
進めないよバーカ
*piece= 1
way= [-1, 1]
[True, True]
line= second
l_next_position= [2, 1]
distance= first
*point= 12
way= [0, 1]
[True, True]
line= second
l_next_position= [1, 1]
distance= first
*point= 12
way= [1, 1]
[True, True]
もうおるやんけ！
robablepos 0
robablepos 2
robable_piece= 2
line= second
l_next_position= [0, 1]
distance= first
*駒取った+point= 24
way= [1, 0]
[True, True]
もうおるやんけ！
robablepos 0
robablepos 2
robablepos 4
味方やんけ
way= [1, -1]
[True, False]
進めないよバーカ
way= [0, -1]
[True, False]
進めないよバーカ
way= [-1, -1]
[True, False]
進めないよバーカ
way= [-1, 0]
[True, True]
もうおるやんけ！
robablepos 0
robablepos 2
robablepos 4
味方やんけ
*piece= 6
way= [-1, 2]
[False, True]
進めないよバーカ
way= [1, 2]
[True, True]
もうおるやんけ！
robablepos 0
味方やんけ
('points=', [[3, 16, [1, 1]], [1, 12, [2, 1]], [1, 12, [1, 1]], [1, 24, [0, 1]]])
